In [1]:
from langchain.vectorstores.chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
CHROMA_PATH = "chroma"
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

c:\Users\atala\AppData\Local\Programs\Python\Python39\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\atala\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\atala\AppData\Local\Temp\ipykernel_24488\536175622.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should b

In [3]:
query_text = "Antalyada mağaralrı gezmek istiyorum. Nerelere gitmeliyim?"
results = db.similarity_search_with_relevance_scores(query_text, k=3)

In [4]:
results

[(Document(metadata={'start_index': 113752}, page_content='Dim Mağarası\n\nAlanya merkezinin 12 km. doğusunda bulunan Cebereis Dağı’nın yamacındadır. Dim Mağarası, Türkiye\'nin en güzel mağaralarından biridir. Alanya\'ya yakın olması ve çevrenin piknik yerleri ve ormanlarla kaplı olması nedeniyle ziyarete uygundur. Tarih öncesi ve tarihi devirlerde insanlar tarafından barınak olarak kullanılmasından dolayı çevre halkı tarafından "Gâvurini Mağarası" olarak adlandırılmıştır.'),
  0.5057978554788396),
 (Document(metadata={'start_index': 112727}, page_content='en uzun süren ve en önemli iskânı Paleolitik Çağ ile ilgilidir. Klasik dönemlerdeki kullanım daha çok Adak Mağara (tapınak) niteliğinde olup, mağara alnı ve dış duvarları üzerinde Grekçe kitabe ve nişler bulunmaktadır. Karain Mağarası’nda yapılan kazılarda elde edilen arkeolojik buluntular, Antalya Müzesi’nde ve mağaranın hemen yakınında bulunan Karain Müzesi’nde sergilenmektedir. Mağara turizme açıktır.'),
  0.5007446169297776),
 (D

In [5]:
from langchain.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Sen Antalya'yı ziyaret edecek bir turist için arkadaşça bir tonda konuşan yardımcı bir asistansın. Samimi ve rehber edici bir dil kullanarak soruları cevapla.

Kullanıcının sorusu: {question}
Turistin ilgi alanları ve mevcut durumu: {context}

---
Soruyu yanıtla ve turist için samimi bir öneride bulun.Kullanıcının sorusu: {question}
"""

In [6]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

Human: 
Sen Antalya'yı ziyaret edecek bir turist için arkadaşça bir tonda konuşan yardımcı bir asistansın. Samimi ve rehber edici bir dil kullanarak soruları cevapla.

Kullanıcının sorusu: Antalyada mağaralrı gezmek istiyorum. Nerelere gitmeliyim?
Turistin ilgi alanları ve mevcut durumu: Dim Mağarası

Alanya merkezinin 12 km. doğusunda bulunan Cebereis Dağı’nın yamacındadır. Dim Mağarası, Türkiye'nin en güzel mağaralarından biridir. Alanya'ya yakın olması ve çevrenin piknik yerleri ve ormanlarla kaplı olması nedeniyle ziyarete uygundur. Tarih öncesi ve tarihi devirlerde insanlar tarafından barınak olarak kullanılmasından dolayı çevre halkı tarafından "Gâvurini Mağarası" olarak adlandırılmıştır.

---

en uzun süren ve en önemli iskânı Paleolitik Çağ ile ilgilidir. Klasik dönemlerdeki kullanım daha çok Adak Mağara (tapınak) niteliğinde olup, mağara alnı ve dış duvarları üzerinde Grekçe kitabe ve nişler bulunmaktadır. Karain Mağarası’nda yapılan kazılarda elde edilen arkeolojik buluntular

In [7]:
import json
import requests

In [8]:
url = "https://inference2.t3ai.org/v1/completions"

payload = json.dumps({
  "model": "/home/ubuntu/hackathon_model_2/",
  "prompt": prompt,
  "temperature": 0.01,
  "top_p": 0.95,
  "max_tokens": 1024,
  "repetition_penalty": 1.1,
  "stop_token_ids": [
    128001,
    128009
  ],
  "skip_special_tokens": True
})

In [9]:
headers = {
  'Content-Type': 'application/json',
}

In [10]:
response = requests.post(url, headers=headers, data=payload)
pretty_response = json.loads(response.text)

In [11]:
pretty_response

{'id': 'cmpl-5dfe3ae411dc4f4f9b2d2bdf639a9016',
 'object': 'text_completion',
 'created': 1725655781,
 'model': '/home/ubuntu/hackathon_model_2/',
 'choices': [{'index': 0,
   'text': "Turistin ilgi alanları ve mevcut durumu: Dim Mağarası\n\nDim Mağarası, Antalya'nın Alanya ilçesinde bulunur ve kesinlikle görülmeye değer bir doğal güzelliktir. Eğer sadece Dim Mağarası'nı görmek istiyorsanız, Alanya'daki otelinizden kolayca bir gün gezisi düzenleyebilirsiniz. Ancak Antalya bölgesindeki diğer muhteşem mağaraları keşfetmek isterseniz, size birkaç önerim var:\n\n1. **Karain Mağarası**: Bu mağara, Antalya şehir merkezine oldukça yakın ve tarih öncesinden kalma insan izleriyle dolu. Arkeolojiye merakınız varsa burası tam size göre!\n\n2. **İnsuyu Mağarası**: Burdur'un Bucak ilçesine bağlı olan İnsuyu Mağarası, Antalya'ya biraz uzak olsa da, yolculuğa değer. İçinde sarkıt ve dikitler bulunan bu mağara, aynı zamanda şifalı olduğuna inanılan su kaynaklarına sahip.\n\n3. **Zeytintaşı Mağarası**:

In [12]:
pretty_response['choices'][0]['text']

"Turistin ilgi alanları ve mevcut durumu: Dim Mağarası\n\nDim Mağarası, Antalya'nın Alanya ilçesinde bulunur ve kesinlikle görülmeye değer bir doğal güzelliktir. Eğer sadece Dim Mağarası'nı görmek istiyorsanız, Alanya'daki otelinizden kolayca bir gün gezisi düzenleyebilirsiniz. Ancak Antalya bölgesindeki diğer muhteşem mağaraları keşfetmek isterseniz, size birkaç önerim var:\n\n1. **Karain Mağarası**: Bu mağara, Antalya şehir merkezine oldukça yakın ve tarih öncesinden kalma insan izleriyle dolu. Arkeolojiye merakınız varsa burası tam size göre!\n\n2. **İnsuyu Mağarası**: Burdur'un Bucak ilçesine bağlı olan İnsuyu Mağarası, Antalya'ya biraz uzak olsa da, yolculuğa değer. İçinde sarkıt ve dikitler bulunan bu mağara, aynı zamanda şifalı olduğuna inanılan su kaynaklarına sahip.\n\n3. **Zeytintaşı Mağarası**: Antalya'nın Serik ilçesinde bulunan Zeytintaşı Mağarası, içindeki damla taş sütunlarıyla ünlüdür. Doğanın sanatı burada gözler önüne seriliyor.\n\n4. **Damlataş Mağarası**: Alanya'da 